### Decorator Application: Single Dispatch Generic Functions

Рассмотрим приложение, в котором мы хотим предоставить похожую функциональность, но которая немного отличается в зависимости от типов переданных аргументов.

В этом наборе примеров мы рассматриваем эту проблему, в которой функциональность отличается в зависимости от типа одного аргумента (следовательно, одиночная диспетчеризация) вместо типа нескольких аргументов (что было бы множественной диспетчеризацией)

Если у вас есть опыт работы с другими объектно-ориентированными языками, такими как Java или C#, вы знаете, что мы можем легко сделать что-то вроде этого, просто **перегружая** функции: используя другой тип данных для параметра функции, тем самым изменяя сигнатуру функции. Тогда, хотя имя функции одинаковое, вызов `do_something(100)` и `do_something('java')` вызовет другую функцию, первая вызовет функцию `do_something(int)`, а вторая вызовет функцию `do_something(String)`.

Конечно, Python не является статически типизированным, поэтому даже если бы в Python была встроена перегрузка функций, мы бы не смогли провести такое различие в сигнатурах наших функций, поскольку нет ничего, что говорило бы о том, что параметр должен иметь определенный тип, поэтому в лучшем случае нам пришлось бы «различать» функции с одинаковым именем только по количеству принимаемых ими параметров. И тогда нам придется как-то иметь дело с переменным числом позиционных и ключевых аргументов... Ууух!
В любом случае, одиночная отправка никогда не сработает.

Вместо этого нам придется придумать другое решение.

Допустим, мы хотим отобразить различные типы данных в формате HTML с различными представлениями для целых чисел (нам нужны как десятичное, так и шестнадцатеричное значения), чисел с плавающей точкой (мы всегда хотим округлять их до двух десятичных знаков), строк (мы хотим, чтобы строка была экранирована HTML, а все символы новой строки заменены на `<br/>`), списки и кортежи должны быть реализованы с использованием маркированных списков, и то же самое со словарями, за исключением того, что мы хотим, чтобы пара имя/значение отображалась в маркированном списке.

Для начала давайте просто реализуем отдельные функции для выполнения каждой из этих задач.

Я собираюсь сделать функции максимально простыми, но на практике вам придется обрабатывать такие ситуации, как объекты None, пустые списки и словари, возможная передача функции неправильного типа и т. д.

In [1]:
from html import escape

def html_escape(arg):
    return escape(str(arg))

def html_int(a):
    return '{0}(<i>{1}</i)'.format(a, str(hex(a)))

def html_real(a):
    return '{0:.2f}'.format(round(a, 2))

def html_str(s):
    return html_escape(s).replace('\n', '<br/>\n')

def html_list(l):
    items = ('<li>{0}</li>'.format(html_escape(item))
             for item in l)
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

def html_dict(d):
    items = ('<li>{0}={1}</li>'.format(html_escape(k), html_escape(v))
             for k, v in d.items())
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [2]:
print(html_str("""this is
a multi line string
with special characters: 10 < 100"""))

this is <br/>
a multi line string<br/>
with special characters: 10 &lt; 100


In [3]:
print(html_int(255))

255(<i>0xff</i)


In [4]:
print(html_escape(3+10j))

(3+10j)


For starters, let's just implement individual functions to do each of those things.

I am going to keep the functions very simple, but in practice you should handle situations like None objects, empty lists and dictionaries, possibly the wrong type being passed to the function, etc.

Мы могли бы попробовать сделать это следующим образом:

In [5]:
from decimal import Decimal

def htmlize(arg):
    if isinstance(arg, int):
        return html_int(arg)
    elif isinstance(arg, float) or isinstance(arg, Decimal):
        return html_real(arg)
    elif isinstance(arg, str):
        return html_str(arg)
    elif isinstance(arg, list) or isinstance(arg, tuple):
        return html_list(arg)
    elif isinstance(arg, dict):
        return html_dict(arg)
    else:
        # default behavior - just html escape string representation
        return html_escape(str(arg))

Теперь мы можем по сути использовать один и тот же вызов функции для обработки различных типов — функция `htmlize` является диспетчером — она направляет запрос другой функции в зависимости от типа аргумента. (Есть гораздо лучший способ сделать часть этого, но для этого нам придется подождать, пока мы не рассмотрим абстрактные базовые классы).

In [6]:
print(htmlize([1, 2, 3]))

<ul>
<li>1</li>
<li>2</li>
<li>3</li>
</ul>


In [7]:
print(htmlize(dict(key1=1, key2=2)))

<ul>
<li>key1=1</li>
<li>key2=2</li>
</ul>


In [8]:
print(htmlize(255))

255(<i>0xff</i)


Но здесь есть ряд недостатков:

In [9]:
print(htmlize(["""first element is
a multi-line string""", (1, 2, 3)]))

<ul>
<li>first element is 
a multi-line string</li>
<li>(1, 2, 3)</li>
</ul>


Как вы можете видеть, в многострочной строке не были заменены символы новой строки, кортеж не был отображен как список html, а целые числа не имеют своего шестнадцатеричного представления.

Поэтому нам просто нужно переопределить функции `html_list` и `html_dict`, чтобы использовать функцию `htmlize`:

In [10]:
def html_list(l):
    items = ['<li>{0}</li>'.format(htmlize(item)) for item in l]
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [11]:
def html_dict(d):
    items = ['<li>{0}={1}</li>'.format(html_escape(k), htmlize(v)) for k, v in d.items()]
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [12]:
print(htmlize(["""first element is
a multi-line string""", (1, 2, 3)]))

<ul>
<li>first element is <br/>
a multi-line string</li>
<li><ul>
<li>1(<i>0x1</i)</li>
<li>2(<i>0x2</i)</li>
<li>3(<i>0x3</i)</li>
</ul></li>
</ul>


Намного лучше, но, надеюсь, вы заметили что-то, что может показаться проблемным!

Разве у нас нет циклической ссылки?

Чтобы определить `html_list` и `html_dict`, нам нужно было вызвать `htmlize`, но чтобы определить `htmlize`, нам нужно было вызвать `html_list` и `html_dict`.

Помните, что в Python мы можем ссылаться на функцию **внутри** тела другой функции **до** определения функции, при условии, что к моменту **вызова** первой функции вторая функция уже определена. ТАК ЧТО это на самом деле нормально.

Если вы мне не верите и хотите убедиться в этом сами, сбросьте свое ядро ​​(щелкните по пункту меню Ядро | Перезапуск) и запустите следующий код, ничего не запуская перед этим.

Тело функции `htmlize` вызывает другие функции, такие как `html_escape`, `html_int` и т. д., которые на самом деле еще не определены.

In [1]:
from html import escape
from decimal import Decimal

def htmlize(arg):
    if isinstance(arg, int):
        return html_int(arg)
    elif isinstance(arg, float) or isinstance(arg, Decimal):
        return html_real(arg)
    elif isinstance(arg, str):
        return html_str(arg)
    elif isinstance(arg, list) or isinstance(arg, tuple) or isinstance(arg, set):
        return html_list(arg)
    elif isinstance(arg, dict):
        return html_dict(arg)
    else:
        # default behavior - just html escape string representation
        return html_escape(str(arg))

Теперь мы определяем все функции, которые использует `htmlize`, прежде чем мы фактически вызовем `htmlize`, и все хорошо:

In [14]:
def html_escape(arg):
    return escape(str(arg))

def html_int(a):
    return '{0}(<i>{1}</i)'.format(a, str(hex(a)))

def html_real(a):
    return '{0:.2f}'.format(round(a, 2))

def html_str(s):
    return html_escape(s).replace('\n', '<br/>\n')

def html_list(l):
    items = ['<li>{0}</li>'.format(htmlize(item)) for item in l]
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

def html_dict(d):
    items = ['<li>{0}={1}</li>'.format(html_escape(k), htmlize(v)) for k, v in d.items()]
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [15]:
print(htmlize(["""first element is
a multi-line string""", (1, 2, 3)]))

<ul>
<li>first element is <br/>
a multi-line string</li>
<li><ul>
<li>1(<i>0x1</i)</li>
<li>2(<i>0x2</i)</li>
<li>3(<i>0x3</i)</li>
</ul></li>
</ul>


Как видите, это работает просто отлично.

Но у нас все еще есть кое-что нежелательное. Вы заметите, что диспетчерская функция `htmlize` должна иметь этот большой оператор `if...elif...else`, который будет продолжать расти, поскольку нам нужно обрабатывать все больше и больше типов (включая потенциально пользовательские типы).

Это просто станет громоздким и не очень гибким (каждый раз, когда кто-то создает новый тип, который должен иметь специальное представление html, ему нужно будет перейти в функцию `htmlize` и изменить ее.

Поэтому вместо этого мы попробуем более гибкий подход с использованием декораторов.

Мы собираемся подойти к этому вопросу следующим образом: создадим функцию-диспетчер, а затем отдельно «зарегистрируем» каждую типоспецифическую функцию в диспетчере.

Сначала мы создадим декоратор, который будет делать что-то, что может показаться немного глупым — он возьмет декорированную функцию и сохранит ее в словаре, используя ключ, состоящий из **type** `object`.

Затем, когда будет вызвано возвращенное замыкание, замыкание вызовет функцию, сохраненную в этом словаре.

In [16]:
def singledispatch(fn):
    registry = dict()
    registry[object] = fn

    def inner(arg):
        return registry[object](arg)

    return inner

In [17]:
@singledispatch
def htmlizer(arg):
    return escape(str(arg))

In [18]:
htmlizer('a < 10')

'a &lt; 10'

Далее мы добавим несколько функций в этот словарь `registry` и изменим нашу внутреннюю функцию, чтобы выбрать правильную функцию из реестра или выбрать функцию по умолчанию на основе типа аргумента:

In [19]:
def singledispatch(fn):
    registry = dict()

    registry[object] = fn
    registry[int] = lambda arg: '{0}(<i>{1}</i)'.format(arg, str(hex(arg)))
    registry[float] = lambda arg: '{0:.2f}'.format(round(arg, 2))

    def inner(arg):
        fn = registry.get(type(arg), registry[object])
        return fn(arg)
    return inner

In [20]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [21]:
htmlize(10)

'10(<i>0xa</i)'

In [22]:
htmlize(3.1415)

'3.14'

Теперь нам нужен способ добавления специализированных функций в словарь `registry` **извне** функции `singledispatch` — для этого мы создадим параметризованный декоратор, который будет (1) принимать тип в качестве параметра и (2) возвращать замыкание, которое будет декорировать функцию, связанную с типом:

In [23]:
def singledispatch(fn):
    registry = dict()

    registry[object] = fn

    def register(type_):
        def inner(fn):
            registry[type_] = fn
        return inner


    def decorator(arg):
        fn = registry.get(type(arg), registry[object])
        return fn(arg)

    return decorator

Но, конечно, этого недостаточно - как нам получить доступ к функции `register` извне `singledispatch`? Помните, `singledispatch` - это декоратор, который возвращает `decorated` замыкание, а не `register` замыкание.

Мы можем сделать это, добавив функцию `register` как **атрибут** функции `decorated` перед тем, как мы ее вернем.

Пока мы этим занимаемся, мы также собираемся:

* добавить словарь `registry` как атрибут, чтобы мы могли заглянуть в него и увидеть, что он содержит.

* добавить еще одну функцию, которая при заданном типе будет возвращать функцию, связанную с этим типом (или функцию по умолчанию, если тип не найден в словаре)

In [24]:
def singledispatch(fn):
    registry = dict()

    registry[object] = fn

    def register(type_):
        def inner(fn):
            registry[type_] = fn
            return fn  # we do this so we can stack register decorators!
        return inner

    def decorator(arg):
        fn = registry.get(type(arg), registry[object])
        return fn(arg)

    def dispatch(type_):
        return registry.get(type_, registry[object])

    decorator.register = register
    decorator.registry = registry.keys()
    decorator.dispatch = dispatch
    return decorator

In [25]:
@singledispatch
def htmlize(arg):
    return escape(str(arg))

И мы видим, что функция `htmlize` (которая вернула `inner`) имеет атрибут, называемый `register`:

In [26]:
htmlize.register

<function __main__.singledispatch.<locals>.register>

а также атрибут `registry`, который мы ввели только для того, чтобы увидеть, какие ключи находятся в словаре `registry`:

In [27]:
htmlize.registry

dict_keys([<class 'object'>])

Мы также можем спросить его, какую функцию он собирается использовать для любого конкретного типа (в настоящее время у нас зарегистрирована только одна функция, используемая по умолчанию, для самого общего типа `object`):

In [28]:
htmlize.dispatch(str)

object

And you'll note that the extended scope of `register` and `dispatch` is the same as the extended scope of `htmlize`.

И вы заметите, что расширенная область действия `register` и `dispatch` такая же, как расширенная область действия `htmlize`.

In [29]:
@htmlize.register(int)
def html_int(a):
    return '{0}(<i>{1}</i)'.format(a, str(hex(a)))

Мы можем заглянуть в зарегистрированные типы:

In [30]:
htmlize.registry

dict_keys([<class 'object'>, <class 'int'>])

and we can ask the decorated `htmlize` function what function it is going to use for the `int` type:

In [31]:
htmlize.dispatch(int)

<function __main__.html_int>

и мы на самом деле можем назвать это так:

In [32]:
htmlize(100)

'100(<i>0x64</i)'

Огромным преимуществом теперь является то, что мы можем продолжать регистрировать новые обработчики из любой точки нашего модуля или даже из-за его пределов!

In [33]:
@htmlize.register(float)
def html_real(a):
    return '{0:.2f}'.format(round(a, 2))

@htmlize.register(str)
def html_str(s):
    return escape(s).replace('\n', '<br/>\n')

@htmlize.register(tuple)
@htmlize.register(list)
def html_list(l):
    items = ['<li>{0}</li>'.format(htmlize(item)) for item in l]
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

@htmlize.register(dict)
def html_dict(d):
    items = ['<li>{0}={1}</li>'.format(htmlize(k), htmlize(v)) for k, v in d.items()]
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [34]:
htmlize.registry

dict_keys([<class 'object'>, <class 'int'>, <class 'float'>, <class 'str'>, <class 'list'>, <class 'tuple'>, <class 'dict'>])

In [35]:
print(htmlize([1, 2, 3]))

<ul>
<li>1(<i>0x1</i)</li>
<li>2(<i>0x2</i)</li>
<li>3(<i>0x3</i)</li>
</ul>


In [36]:
print(htmlize((1, 2, 3)))

<ul>
<li>1(<i>0x1</i)</li>
<li>2(<i>0x2</i)</li>
<li>3(<i>0x3</i)</li>
</ul>


In [37]:
print(htmlize("""this
is a multi line string with
a < 10"""))

this<br/>
is a multi line string with<br/>
a &lt; 10


Наш декоратор с одной отправкой работает довольно хорошо, но у него есть некоторые ограничения. Например, он не может обрабатывать функции, которые принимают более одного аргумента (в этом случае отправка будет основана на типе **первого** аргумента), и мы также не допускаем типы, основанные на родительских классах, например, целые числа и логические значения являются целыми числами, т. е. они оба наследуются от базового класса Integral. Аналогично списки и кортежи являются более общими типами Sequence. Мы рассмотрим это более подробно, когда перейдем к теме абстрактных базовых классов (ABC).

In [38]:
from numbers import Integral

In [39]:
isinstance(100, Integral)

True

In [40]:
isinstance(True, Integral)

True

In [41]:
isinstance(100.5, Integral)

False

In [42]:
type(100) is Integral

False

In [43]:
type(True) is Integral

False

In [44]:
(100).__class__

int

In [45]:
(True).__class__

bool

Способ, которым мы реализовали наш декоратор, заключается в том, что если мы регистрируем обобщенную функцию Integral, она не будет принимать ни целые числа, ни булевы значения.

Мы, конечно, можем исправить этот недостаток самостоятельно, но, конечно...

Мы можем использовать встроенную в Python поддержку одиночной диспетчеризации, в ...

вы угадали!

модуле `functools`.

In [46]:
from functools import singledispatch
from numbers import Integral
from collections.abc import Sequence

In [47]:
@singledispatch
def htmlize(a):
    return escape(str(a))

Возвращаемое замыкание `singledispatch` имеет несколько атрибутов, которые мы можем использовать:
1. Декоратор `register` (как у нас)
2. Свойство `registry`, которое является словарем реестра
3. Функция `dispatch`, которая может использоваться для определения того, какой раздел реестра (зарегистрированный тип) будет использоваться для указанного типа.

In [48]:
@htmlize.register(Integral)
def htmlize_int(a):
    return '{0}(<i>{1}</i)'.format(a, str(hex(a)))

In [49]:
htmlize.dispatch(int)

<function __main__.htmlize_int>

In [50]:
htmlize.dispatch(bool)

<function __main__.htmlize_int>

In [51]:
htmlize(100)

'100(<i>0x64</i)'

In [52]:
htmlize(True)

'True(<i>0x1</i)'

In [53]:
@htmlize.register(Sequence)
def html_sequence(l):
    items = ['<li>{0}</li>'.format(htmlize(item)) for item in l]
    return '<ul>\n' + '\n'.join(items) + '\n</ul>'

In [54]:
htmlize.dispatch(list)

<function __main__.html_sequence>

In [55]:
htmlize.dispatch(tuple)

<function __main__.html_sequence>

In [56]:
htmlize.dispatch(str)

<function __main__.html_sequence>

Вы заметите, что строка также является типом последовательности, поэтому наш диспетчер вызовет функцию `html_sequence` для строки.

На самом деле, в этот момент все даже не будет работать должным образом.

Если бы мы вызвали

`htmlize('abc')`

мы бы получили бесконечную рекурсию!

Вызов `htmlize` строки `abc` обработал бы ее как последовательность, что вызвало бы `htmlize` символ за символом. Но каждый символ сам по себе является просто строкой длиной 1, поэтому он вызовет `htmlize` для этого одного символа, что обработает его как последовательность, что снова вызовет `htmlize` для этого одного символа, и так далее, в бесконечном цикле.

In [57]:
htmlize('abc')

RecursionError: maximum recursion depth exceeded

Вместо этого мы зарегистрируем обработчик строк специально — таким образом мы полностью избежим этой проблемы:

In [58]:
@htmlize.register(str)
def html_str(s):
    return escape(s).replace('\n', '<br/>\n')

In [59]:
htmlize.dispatch(str)

<function __main__.html_str>

Таким образом, даже если строка является как экземпляром `str`, так и в целом типом последовательности, диспетчер выберет «ближайший» тип (опять же, чего не сделала наша собственная реализация).

Это означает, что у нас есть что-то для общих последовательностей, но что-то конкретное для более специализированных строк.

In [60]:
htmlize('abc')

'abc'

То же самое мы можем сделать с последовательностями — сейчас `html_sequence` будет использоваться как для списков, так и для кортежей.

Но предположим, что нам нужна немного иная обработка кортежей:

In [61]:
@htmlize.register(tuple)
def html_tuple(t):
    items = [escape(str(item)) for item in t]
    return '({0})'.format(', '.join(items))

In [62]:
htmlize.dispatch(list)

<function __main__.html_sequence>

In [63]:
htmlize.dispatch(tuple)

<function __main__.html_tuple>

In [64]:
print(htmlize(['a', 100, 3.14]))

<ul>
<li>a</li>
<li>100(<i>0x64</i)</li>
<li>3.14</li>
</ul>


In [65]:
print(htmlize(('a', 100, 3.14)))

(a, 100, 3.14)


Стоит отметить, что мы начали наше оформление с декоратора `@singledispatch` — вы заметите, что здесь не указан конкретный тип — и на самом деле это означает, что диспетчер будет использовать общий тип `object`.

Это означает, что любой тип объекта, специально не обрабатываемый нашим диспетчером, будет возвращаться к этому ключу `object`, поэтому его можно рассматривать как значение по умолчанию для диспетчера.

In [66]:
type(None)

NoneType

In [67]:
htmlize.dispatch(type(None))

<function __main__.htmlize>

In [68]:
type(1+1j)

complex

In [69]:
htmlize.dispatch(complex)

<function __main__.htmlize>

In [70]:
type(3)

int

In [71]:
htmlize.dispatch(int)

<function __main__.htmlize_int>

Наконец, поскольку имя отдельной специализированной функции не имеет для нас особого значения (диспетчер выберет соответствующую функцию), для имени функции довольно часто используется символ подчеркивания ( \_ ) — адрес памяти каждой специализированной функции будет сохранен в словаре `registry`, а имя функции не имеет значения — на самом деле мы даже можем добавлять лямбда-выражения в реестр.

In [72]:
@singledispatch
def htmlize(a):
    return escape(str(a))

In [73]:
@htmlize.register(int)
def _(a):
    return '{0}({1})'.format(a, str(hex(a)))

In [74]:
@htmlize.register(str)
def _(s):
    return escape(s).replace('\n', '<br/>\n')

In [75]:
htmlize.register(float)(lambda f: '{0:.2f}'.format(f))

<function __main__.<lambda>>

In [76]:
htmlize.registry

mappingproxy({object: <function __main__.htmlize>,
              int: <function __main__._>,
              str: <function __main__._>,
              float: <function __main__.<lambda>>})

Но обратите внимание, что функция `__main__._` для `int` и `str` — это не одни и те же функции (даже если у них одинаковые имена):

In [77]:
id(htmlize.registry[str])

3104966916432

In [78]:
id(htmlize.registry[int])

3104967451784

And everything works as expected:

In [79]:
htmlize(100)

'100(0x64)'

In [80]:
htmlize(3.1415)

'3.14'

In [81]:
print(htmlize("""this
is a multi-line string
a < 10"""))

this<br/>
is a multi-line string<br/>
a &lt; 10


Если вас смущает одно и то же название, но разные функции, посмотрите на это так:

In [82]:
def my_func():
    print('my_func initial')

In [83]:
id(my_func)

3104966916296

In [84]:
f = my_func

In [85]:
id(f)

3104966916296

Итак, `f` и `my_func` указывают на одну и ту же функцию в памяти.

Давайте продолжим и «переопределим» функцию `my_func`:

In [86]:
def my_func():
    print('second my_func')

На самом деле, мы не «переопределяли» предыдущую `my_func`, она все еще существует в памяти (и `f` все еще указывает на нее). Вместо этого мы переназначили функцию, на которую указывает `my_func`:

In [87]:
id(my_func)

3104966914800

Но исходный `my_func` все еще существует, и `f' все еще имеет ссылку на него:

In [88]:
id(f)

3104966916296

Итак, мы можем назвать каждый из них:

In [89]:
f()

my_func initial


In [90]:
my_func()

second my_func


Но функция `__name__` имеет то же значение:

In [91]:
f.__name__

'my_func'

In [92]:
my_func.__name__

'my_func'

Просто всегда помните, что метки указывают на что-то в памяти, а не сам объект. Так что в этом случае у нас есть два отдельных объекта (функции), которые имеют одинаковое имя, но являются двумя совершенно разными объектами - `f` указывает на первый, который мы создали, а `my_func` указывает на второй.